# medium-articles — demo notebook\nThis notebook demonstrates a minimal, test-driven workflow for the "proceed with 1" teaching exercise.\n\n[Open in Colab](https://colab.research.google.com/)  
**What this notebook does:** implement a tiny function, add tests, run pytest, capture output, and produce a git patch.

In [ ]:
# 1 — Environment & dependencies
import sys
import subprocess
from importlib import import_module

print('python:', sys.version.splitlines()[0])
try:
    import pytest
    print('pytest:', pytest.__version__)
except Exception as exc:
    print('pytest not available in the environment:', exc)

print('\nRecommended (local):')
print('python -m venv .venv && . .venv/bin/activate')
print('pip install -r examples/intro-python/requirements.txt')

In [ ]:
# 2 — Inspect active document & repository context
from pathlib import Path
import subprocess

REPO_ROOT = Path('..').resolve() if Path('.git').exists() is False else Path('.').resolve()
ACTIVE_FILE = Path('examples/intro-python/solution.py')

print('repo root:', REPO_ROOT)
print('\nListing relevant files:')
for p in ['articles/01-intro-to-topic.md', 'examples/intro-python/solution.py', 'notebooks/intro-demo.ipynb']:
    print('-', p, '->', Path(p).exists())

print('\nFirst 200 chars of ACTIVE_FILE (if present):')
if ACTIVE_FILE.exists():
    print(ACTIVE_FILE.read_text()[:200])
else:
    print('ACTIVE_FILE not found:', ACTIVE_FILE)

# git info
for cmd in [['git','rev-parse','--abbrev-ref','HEAD'], ['git','status','--porcelain']]:
    try:
        out = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print('\n$', ' '.join(cmd), '\n', out.stdout.strip())
    except Exception as e:
        print('\n$', ' '.join(cmd), '\n', 'git not available or not a repo in the notebook environment:', e)

## 3 — Implement "Procceed with 1" — minimal runnable

We implement a tiny, deterministic function `step1(n) -> n + 1` in `examples/step1.py`. The notebook writes the file so students can run tests and iterate.

In [ ]:
# write the minimal implementation to examples/step1.py
from pathlib import Path
impl_path = Path('examples/step1.py')
impl_path.parent.mkdir(parents=True, exist_ok=True)
impl_path.write_text('''"""Minimal step-1 implementation for classroom demo."""\nfrom typing import Any\n\ndef step1(n: int) -> int:\n    """Return n + 1 (pure, deterministic)."""\n    if not isinstance(n, int):\n        raise TypeError('n must be int')\n    return n + 1\n\n# quick sanity check\nassert step1(1) == 2\n''')
print('Wrote', impl_path)
print(impl_path.read_text())

In [ ]:
# 4 — Examples & unit tests for step 1\nfrom pathlib import Path\ntests_path = Path('examples/intro-python/tests/test_step1.py')\ntests_path.parent.mkdir(parents=True, exist_ok=True)\ntests_path.write_text('''import pytest\nfrom examples.step1 import step1\n\n@pytest.mark.parametrize("inp,expected", [(0,1),(1,2),(99,100)])\ndef test_step1_normal(inp, expected):\n    assert step1(inp) == expected\n\ndef test_step1_typeerror():\n    with pytest.raises(TypeError):\n        step1('x')\n\n# Intentionally failing test to demonstrate debugging (student should fix/remove)\ndef test_intentional_failure():\n    assert step1(2) == 5\n''')\nprint('Wrote tests to', tests_path)\nprint(tests_path.read_text())

In [ ]:
# 5 — Run tests (from the notebook)\nimport subprocess, textwrap\n\ncmd = ['pytest', '-q', 'examples/intro-python/tests/test_step1.py']\nproc = subprocess.run(cmd, capture_output=True, text=True)\nprint('EXIT', proc.returncode)\nprint('\n--- STDOUT ---\n')\nprint(proc.stdout)\nprint('\n--- STDERR ---\n')\nprint(proc.stderr)\n\n# Programmatic run (pytest.main) - capture results in-memory\nimport pytest\nres = pytest.main(['-q', 'examples/intro-python/tests/test_step1.py'])\nprint('pytest.main exit code:', res)

In [ ]:
# 6 — Capture VS Code integrated output (example)
import subprocess
p = subprocess.run(['pytest','-q','examples/intro-python/tests/test_step1.py'], capture_output=True, text=True)
print('returncode=', p.returncode)
print('stdout snippet:\n', p.stdout[:1000])
print('stderr snippet:\n', p.stderr[:1000])

# The VS Code Test Explorer maps pytest output to test cases; the above stdout shows failing tests and tracebacks.

In [ ]:
# 7 — Create git patch & open a PR (local demo)
import subprocess, os
branch = 'feat/proceed-1'
try:
    subprocess.run(['git','checkout','-b',branch], check=True)
except Exception as e:
    print('git checkout failed (maybe already on branch):', e)
# stage the new files we created in the notebook
subprocess.run(['git','add','examples/step1.py','examples/intro-python/tests/test_step1.py'], check=False)
subprocess.run(['git','commit','-m','impl: proceed with 1 (minimal)'], check=False)
# produce a patch against origin/main (if available)
try:
    out = subprocess.run(['git','diff','origin/main...HEAD','--stat'], capture_output=True, text=True)
    print(out.stdout)
    patch = subprocess.run(['git','format-patch','origin/main','--stdout'], capture_output=True, text=True)
    Path('proceed-1.patch').write_text(patch.stdout)
    print('\nWrote proceed-1.patch (preview)')
except Exception as e:
    print('Could not create patch (no remote/main available):', e)

print('\nTo push and open a PR:')
print(f'  git push --set-upstream origin {branch}')
print('  Open a PR from your fork on GitHub (or use `gh pr create`)')

## Next steps & teaching tips
- Ask students to remove the intentionally failing test and submit a PR.
- Use GitHub Classroom to create starter repos/assignments from this template.

**Tip:** add a `starter/` branch containing only scaffolded files so students can fork and work from a clean baseline.